# Ground HPO with Optuna (MLP, LSTM, BiLSTM, CNN-LSTM, Transformer)

## Libraries

In [1]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers, backend as K

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from optuna.integration import TFKerasPruningCallback
from optuna.storages import JournalStorage
from optuna.storages import JournalFileStorage, JournalFileOpenLock

2025-09-24 22:06:47.184061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-24 22:06:47.189332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758769607.195480  615932 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758769607.197515  615932 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-24 22:06:47.204541: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Config

In [2]:
SEED = 42
np.random.seed(SEED); tf.random.set_seed(SEED)

DATA_DIR = Path("../data_processed")
OUT_DIR  = Path("../models"); OUT_DIR.mkdir(parents=True, exist_ok=True)
STUDY_DIR= Path("../optuna_studies"); STUDY_DIR.mkdir(parents=True, exist_ok=True)
ART_DIR  = (OUT_DIR / "optuna_artifacts").resolve(); ART_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_PQ = DATA_DIR / "ground_train_h6.parquet"
VAL_PQ   = DATA_DIR / "ground_val_h6.parquet"
TEST_PQ  = DATA_DIR / "ground_test_h6.parquet"
TARGET   = "y_ghi_h6"   # "y_k_h6" para el índice k

print("Studies dir:", STUDY_DIR.resolve())
print("Artifacts dir:", ART_DIR)

Studies dir: /mnt/SOLARLAB/E_Ladino/Repo/irradiance-fusion-forecast/optuna_studies
Artifacts dir: /mnt/SOLARLAB/E_Ladino/Repo/irradiance-fusion-forecast/models/optuna_artifacts


### Data loading and preprocessing

In [3]:
train = pd.read_parquet(TRAIN_PQ).sort_index()
val   = pd.read_parquet(VAL_PQ).sort_index()
test  = pd.read_parquet(TEST_PQ).sort_index()
assert TARGET in train and TARGET in val and TARGET in test, f"{TARGET} missing!"

feat_cols = sorted(list(set(train.columns) & set(val.columns) & set(test.columns) - {TARGET}))
feat_cols = [c for c in feat_cols if pd.api.types.is_numeric_dtype(train[c])]

Xtr_df, ytr = train[feat_cols], train[TARGET]
Xva_df, yva = val[feat_cols],   val[TARGET]
Xte_df, yte = test[feat_cols],  test[TARGET]

scaler = StandardScaler()
Xtr = scaler.fit_transform(Xtr_df)
Xva = scaler.transform(Xva_df)
Xte = scaler.transform(Xte_df)

## Helpers

In [4]:
def _rmse(a,b): 
    return float(np.sqrt(mean_squared_error(a,b)))

def skill(y_true, y_pred, y_base):
    return 1.0 - (_rmse(y_true, y_pred) / _rmse(y_true, y_base))

def _build_seq(X_df, y_ser, L):
    """Secuencias sin índice (rápido para objetivos Optuna)."""
    Xv, yv = X_df.values, y_ser.values
    xs, ys = [], []
    for i in range(L-1, len(X_df)):
        block = Xv[i-L+1:i+1]
        if np.isnan(block).any():
            continue
        xs.append(block); ys.append(yv[i])
    return np.asarray(xs, dtype="float32"), np.asarray(ys, dtype="float32")

def build_seq_with_idx(X_df, y_ser, L):
    """Secuencias con índice (para evaluación y plots)."""
    Xv, yv = X_df.values, y_ser.values
    xs, ys, idx = [], [], []
    for i in range(L-1, len(X_df)):
        block = Xv[i-L+1:i+1]
        if np.isnan(block).any():
            continue
        xs.append(block); ys.append(yv[i]); idx.append(X_df.index[i])
    return (np.asarray(xs, dtype="float32"),
            np.asarray(ys, dtype="float32"),
            pd.DatetimeIndex(idx))

def prepare_journal_storage(study_name: str) -> JournalStorage:
    log_path   = STUDY_DIR / f"{study_name}.log"
    lock_path  = STUDY_DIR / f"{study_name}.lock"
    # limpiar lock si quedó colgado
    try: lock_path.unlink()
    except FileNotFoundError: pass
    file_storage = JournalFileStorage(str(log_path), lock_obj=JournalFileOpenLock(str(lock_path)))
    return JournalStorage(file_storage)

def _safe_load_best(study, rebuild_fn=None):
    """Carga robusta del mejor modelo guardado por el estudio."""
    p = Path(study.best_trial.user_attrs["model_path"])
    if not p.exists():
        # fallback: buscar por nombre
        hits = list(ART_DIR.rglob(p.name))
        if hits:
            p = hits[0]
        elif rebuild_fn is not None:
            model = rebuild_fn(study.best_trial.params)
            p = ART_DIR / "recover.keras"
            model.save(p)
        else:
            raise FileNotFoundError(f"Checkpoint not found: {p}")
    return tf.keras.models.load_model(p), p

## Baseline

In [5]:
base_src = None

# for c in ["k_ghi","k_raw","k_ghi_lag1","k_raw_lag1"]:
#     if c in test.columns: base_src = test[c]; break
# if base_src is None:
#     base_src = pd.Series(np.nanmedian(ytr), index=test.index)


for c in ["ghi_qc","ghi_sg_definitive","ghi_qc_lag1"]:
    if c in test.columns: base_src = test[c]; break
if base_src is None:
    base_src = pd.Series(np.nanmedian(ytr), index=test.index)

y_base = base_src.to_numpy()
print(f"Baseline → RMSE: {_rmse(yte, y_base):.4f} | MAE: {mean_absolute_error(yte, y_base):.4f}")

Baseline → RMSE: 196.2835 | MAE: 102.1871


## Track A - MLP

In [6]:
def objective_mlp(trial: optuna.Trial) -> float:
    K.clear_session()
    n1  = trial.suggest_int("n1", 64, 512, step=64)
    n2  = trial.suggest_int("n2", 32, max(64, n1//2), step=32)
    do1 = trial.suggest_float("do1", 0.0, 0.5)
    do2 = trial.suggest_float("do2", 0.0, 0.5)
    lr  = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    l2w = trial.suggest_float("l2", 1e-8, 1e-3, log=True)
    act = trial.suggest_categorical("act", ["relu","selu","gelu"])
    bs  = trial.suggest_categorical("batch", [64, 128, 256, 512])
    eps = trial.suggest_int("epochs", 40, 150)

    model = models.Sequential([
        layers.Input(shape=(Xtr.shape[1],)),
        layers.Dense(n1, activation=act, kernel_regularizer=regularizers.l2(l2w)),
        layers.Dropout(do1),
        layers.Dense(n2, activation=act, kernel_regularizer=regularizers.l2(l2w)),
        layers.Dropout(do2),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss="mse", metrics=["mae"])

    tmp_dir  = ART_DIR / f"A_mlp_t{trial.number:04d}"; tmp_dir.mkdir(parents=True, exist_ok=True)
    tmp_path = (tmp_dir / "best.keras").resolve()

    cbs = [
        callbacks.EarlyStopping(monitor="val_loss", patience=12, restore_best_weights=True, verbose=0),
        callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6, min_lr=1e-5, verbose=0),
        callbacks.ModelCheckpoint(filepath=str(tmp_path), monitor="val_loss",
                                  save_best_only=True, save_weights_only=False),
        TFKerasPruningCallback(trial, "val_loss"),
    ]

    model.fit(Xtr, ytr, validation_data=(Xva, yva),
              epochs=eps, batch_size=bs, verbose=0, callbacks=cbs)

    yhat = model.predict(Xva, verbose=0).squeeze()
    val_rmse = _rmse(yva, yhat)

    # if not tmp_path.exists():
    #     model.save(tmp_path)

    trial.set_user_attr("model_path", str(tmp_path))
    return val_rmse

In [7]:
storageA = prepare_journal_storage("ground_trackA_mlp")
studyA = optuna.create_study(direction="minimize",
                             sampler=TPESampler(seed=SEED),
                             pruner=MedianPruner(n_startup_trials=8, n_warmup_steps=5),
                             study_name="ground_trackA_mlp",
                             storage=storageA, load_if_exists=True)
print("Running Study A (MLP)…")
studyA.optimize(objective_mlp, n_trials=40, show_progress_bar=True)

best_mlp, bestA_path = _safe_load_best(studyA)
yhatA = best_mlp.predict(Xte, verbose=0).squeeze()
print("Best MLP params:", studyA.best_trial.params)
print(f"MLP test → RMSE: {_rmse(yte, yhatA):.4f} | MAE: {mean_absolute_error(yte, yhatA):.4f} | R2: {r2_score(yte, yhatA):.4f} | Skill: {skill(yte, yhatA, y_base):.3f}")

/tmp/ipykernel_615932/3401579257.py:37: FutureWarning: The import path :class:`~optuna.storages.JournalFileOpenLock` has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileOpenLock` instead.
  file_storage = JournalFileStorage(str(log_path), lock_obj=JournalFileOpenLock(str(lock_path)))
/tmp/ipykernel_615932/3401579257.py:37: FutureWarning: JournalFileStorage has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileBackend` instead.
  file_storage = JournalFileStorage(str(log_path), lock_obj=JournalFileOpenLock(str(lock_path)))
[I 2025-09-24 22:06:49,911] Using an existing study with name 'ground_trackA_mlp' instead of creating a new one.


Running Study A (MLP)…


  0%|          | 0/40 [00:00<?, ?it/s]2025-09-24 22:06:50.273826: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2025-09-24 22:06:50.273845: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: solarlivinglabx
2025-09-24 22:06:50.273848: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: solarlivinglabx
2025-09-24 22:06:50.273904: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 580.65.6
2025-09-24 22:06:50.273915: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 575.64.3
2025-09-24 22:06:50.273918: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:262] kernel version 575.64.3 does not match D

[I 2025-09-24 22:06:52,950] Trial 80 pruned. Trial was pruned at epoch 6.


Best trial: 41. Best value: 129.76:   5%|▌         | 2/40 [00:09<03:08,  4.96s/it]

...

[I 2025-09-24 22:10:32,790] Trial 117 pruned. Trial was pruned at epoch 5.


Best trial: 41. Best value: 129.76:  98%|█████████▊| 39/40 [03:46<00:05,  5.31s/it]

[I 2025-09-24 22:10:36,744] Trial 118 pruned. Trial was pruned at epoch 6.


Best trial: 41. Best value: 129.76: 100%|██████████| 40/40 [03:49<00:00,  5.75s/it]


[I 2025-09-24 22:10:39,824] Trial 119 pruned. Trial was pruned at epoch 5.
Best MLP params: {'n1': 320, 'n2': 128, 'do1': 0.4136533265515615, 'do2': 0.32902271732486804, 'lr': 0.002993580716382224, 'l2': 6.43129160846118e-05, 'act': 'gelu', 'batch': 512, 'epochs': 114}
MLP test → RMSE: 135.6021 | MAE: 66.9260 | R2: 0.7227 | Skill: 0.309


## Track B - Sequentials

### Mods

In [8]:
Xtr_s = pd.DataFrame(Xtr, index=Xtr_df.index, columns=feat_cols)
Xva_s = pd.DataFrame(Xva, index=Xva_df.index, columns=feat_cols)
Xte_s = pd.DataFrame(Xte, index=Xte_df.index, columns=feat_cols)

### LSTM

In [9]:
def objective_lstm(trial: optuna.Trial) -> float:
    K.clear_session()
    L   = trial.suggest_categorical("seq_len", [6, 12, 18, 24])
    u   = trial.suggest_int("units", 32, 128, step=32)
    do  = trial.suggest_float("dropout", 0.0, 0.4)
    lr  = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
    bs  = trial.suggest_categorical("batch", [64, 128, 256])
    eps = trial.suggest_int("epochs", 40, 120)

    Xtr_seq, ytr_seq = _build_seq(Xtr_s, ytr, L)
    Xva_seq, yva_seq = _build_seq(Xva_s, yva, L)
    if min(map(len,[Xtr_seq, Xva_seq])) == 0: 
        raise optuna.TrialPruned()

    model = models.Sequential([
        layers.Input(shape=(L, Xtr_seq.shape[2])),
        layers.LSTM(u, dropout=do),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"])

    tmp_dir  = ART_DIR / f"B_lstm_t{trial.number:04d}"; tmp_dir.mkdir(parents=True, exist_ok=True)
    tmp_path = (tmp_dir / "best.keras").resolve()

    cbs = [
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=0),
        callbacks.ModelCheckpoint(filepath=str(tmp_path), monitor="val_loss",
                                  save_best_only=True, save_weights_only=False),
        TFKerasPruningCallback(trial, "val_loss"),
    ]
    model.fit(Xtr_seq, ytr_seq, validation_data=(Xva_seq, yva_seq),
              epochs=eps, batch_size=bs, verbose=0, callbacks=cbs)

    yhat = model.predict(Xva_seq, verbose=0).squeeze()
    val_rmse = _rmse(yva_seq, yhat)

    if not tmp_path.exists():
        model.save(tmp_path)

    trial.set_user_attr("model_path", str(tmp_path))
    trial.set_user_attr("seq_len_used", L)
    return val_rmse

In [ ]:
storageB1 = prepare_journal_storage("ground_trackB_lstm")
studyB1 = optuna.create_study(direction="minimize",
                              sampler=TPESampler(seed=SEED),
                              pruner=MedianPruner(n_startup_trials=8, n_warmup_steps=5),
                              study_name="ground_trackB_lstm",
                              storage=storageB1, load_if_exists=True)
print("Running Study B1 (LSTM)…")
studyB1.optimize(objective_lstm, n_trials=40, show_progress_bar=True)

best_lstm, _ = _safe_load_best(studyB1)
bestL1 = studyB1.best_trial.user_attrs["seq_len_used"]
Xte_seq, yte_seq, idx_LSTM = build_seq_with_idx(Xte_s, yte, bestL1)
yhatB1 = best_lstm.predict(Xte_seq, verbose=0).squeeze()
y_base_LSTM = pd.Series(y_base, index=Xte_df.index).reindex(idx_LSTM).to_numpy()
print("Best LSTM params:", studyB1.best_trial.params | {"seq_len": bestL1})
print(f"LSTM test → RMSE: {_rmse(yte_seq, yhatB1):.4f} | MAE: {mean_absolute_error(yte_seq, yhatB1):.4f} | R2: {r2_score(yte_seq, yhatB1):.4f} | Skill: {skill(yte_seq, yhatB1, y_base_LSTM):.3f}")

/tmp/ipykernel_615932/3401579257.py:37: FutureWarning: The import path :class:`~optuna.storages.JournalFileOpenLock` has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileOpenLock` instead.
  file_storage = JournalFileStorage(str(log_path), lock_obj=JournalFileOpenLock(str(lock_path)))
/tmp/ipykernel_615932/3401579257.py:37: FutureWarning: JournalFileStorage has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileBackend` instead.
  file_storage = JournalFileStorage(str(log_path), lock_obj=JournalFileOpenLock(str(lock_path)))
[I 2025-09-24 22:10:40,258] Using an existing study with name 'ground_trackB_lstm' instead of creating a new one.


Running Study B1 (LSTM)…


Best trial: 22. Best value: 130.254:   2%|▎         | 1/40 [00:31<20:20, 31.30s/it]

[I 2025-09-24 22:11:11,554] Trial 40 pruned. Trial was pruned at epoch 17.


Best trial: 41. Best value: 130.022:   5%|▌         | 2/40 [01:40<33:58, 53.65s/it]

...

[I 2025-09-24 22:15:27,706] Trial 44 finished with value: 129.939266161638 and parameters: {'seq_len': 18, 'units': 96, 'dropout': 0.33451698668631263, 'lr': 0.002312846347785296, 'batch': 64, 'epochs': 61}. Best is trial 44 with value: 129.939266161638.


Best trial: 44. Best value: 129.939:  15%|█▌        | 6/40 [06:17<40:38, 71.73s/it]

[I 2025-09-24 22:16:57,546] Trial 45 finished with value: 130.20014640928787 and parameters: {'seq_len': 18, 'units': 96, 'dropout': 0.3455174628601367, 'lr': 0.0023043316062673637, 'batch': 64, 'epochs': 61}. Best is trial 44 with value: 129.939266161638.


Best trial: 44. Best value: 129.939:  18%|█▊        | 7/40 [06:36<29:58, 54.51s/it]

[I 2025-09-24 22:17:16,614] Trial 46 pruned. Trial was pruned at epoch 5.


### BiLSTM

In [ ]:
def objective_bilstm(trial: optuna.Trial) -> float:
    K.clear_session()
    L   = trial.suggest_categorical("seq_len", [6, 12, 18, 24])
    u   = trial.suggest_int("units", 32, 128, step=32)
    do  = trial.suggest_float("dropout", 0.0, 0.4)
    lr  = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
    bs  = trial.suggest_categorical("batch", [64, 128, 256])
    eps = trial.suggest_int("epochs", 40, 120)

    Xtr_seq, ytr_seq = _build_seq(Xtr_s, ytr, L)
    Xva_seq, yva_seq = _build_seq(Xva_s, yva, L)
    if min(map(len,[Xtr_seq, Xva_seq])) == 0: 
        raise optuna.TrialPruned()

    model = models.Sequential([
        layers.Input(shape=(L, Xtr_seq.shape[2])),
        layers.Bidirectional(layers.LSTM(u, dropout=do)),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"])

    tmp_dir  = ART_DIR / f"B_bilstm_t{trial.number:04d}"; tmp_dir.mkdir(parents=True, exist_ok=True)
    tmp_path = (tmp_dir / "best.keras").resolve()

    cbs = [
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=0),
        callbacks.ModelCheckpoint(filepath=str(tmp_path), monitor="val_loss",
                                  save_best_only=True, save_weights_only=False),
        TFKerasPruningCallback(trial, "val_loss"),
    ]
    model.fit(Xtr_seq, ytr_seq, validation_data=(Xva_seq, yva_seq),
              epochs=eps, batch_size=bs, verbose=0, callbacks=cbs)

    yhat = model.predict(Xva_seq, verbose=0).squeeze()
    val_rmse = _rmse(yva_seq, yhat)

    if not tmp_path.exists():
        model.save(tmp_path)

    trial.set_user_attr("model_path", str(tmp_path))
    trial.set_user_attr("seq_len_used", L)
    return val_rmse

In [ ]:
storageB2 = prepare_journal_storage("ground_trackB_bilstm")
studyB2 = optuna.create_study(direction="minimize",
                              sampler=TPESampler(seed=SEED),
                              pruner=MedianPruner(n_startup_trials=8, n_warmup_steps=5),
                              study_name="ground_trackB_bilstm",
                              storage=storageB2, load_if_exists=True)
print("Running Study B2 (BiLSTM)…")
studyB2.optimize(objective_bilstm, n_trials=35, show_progress_bar=True)

best_bi, _ = _safe_load_best(studyB2)
bestL2 = studyB2.best_trial.user_attrs["seq_len_used"]
Xte_seq, yte_seq, idx_BI = build_seq_with_idx(Xte_s, yte, bestL2)
yhatB2 = best_bi.predict(Xte_seq, verbose=0).squeeze()
y_base_BI = pd.Series(y_base, index=Xte_df.index).reindex(idx_BI).to_numpy()
print("Best BiLSTM params:", studyB2.best_trial.params | {"seq_len": bestL2})
print(f"BiLSTM test → RMSE: {_rmse(yte_seq, yhatB2):.4f} | MAE: {mean_absolute_error(yte_seq, yhatB2):.4f} | R2: {r2_score(yte_seq, yhatB2):.4f} | Skill: {skill(yte_seq, yhatB2, y_base_BI):.3f}")

### CNN-LSTM

In [ ]:
def objective_cnnlstm(trial: optuna.Trial) -> float:
    K.clear_session()
    L     = trial.suggest_categorical("seq_len", [6, 12, 18, 24])
    filt  = trial.suggest_int("filters", 16, 64, step=16)
    ksz   = trial.suggest_categorical("kernel_size", [2,3,5])
    pool  = trial.suggest_categorical("pool", [1,2])
    u     = trial.suggest_int("lstm_units", 32, 128, step=32)
    do    = trial.suggest_float("dropout", 0.0, 0.4)
    lr    = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
    bs    = trial.suggest_categorical("batch", [64, 128, 256])
    eps   = trial.suggest_int("epochs", 40, 120)

    Xtr_seq, ytr_seq = _build_seq(Xtr_s, ytr, L)
    Xva_seq, yva_seq = _build_seq(Xva_s, yva, L)
    if min(map(len,[Xtr_seq, Xva_seq])) == 0: 
        raise optuna.TrialPruned()

    model = models.Sequential([
        layers.Input(shape=(L, Xtr_seq.shape[2])),
        layers.Conv1D(filt, kernel_size=ksz, padding="causal", activation="relu"),
        layers.MaxPooling1D(pool_size=pool) if pool>1 else layers.Lambda(lambda z: z),
        layers.LSTM(u, dropout=do),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"])

    tmp_dir  = ART_DIR / f"B_cnnlstm_t{trial.number:04d}"; tmp_dir.mkdir(parents=True, exist_ok=True)
    tmp_path = (tmp_dir / "best.keras").resolve()

    cbs = [
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=0),
        callbacks.ModelCheckpoint(filepath=str(tmp_path), monitor="val_loss",
                                  save_best_only=True, save_weights_only=False),
        TFKerasPruningCallback(trial, "val_loss"),
    ]
    model.fit(Xtr_seq, ytr_seq, validation_data=(Xva_seq, yva_seq),
              epochs=eps, batch_size=bs, verbose=0, callbacks=cbs)

    yhat = model.predict(Xva_seq, verbose=0).squeeze()
    val_rmse = _rmse(yva_seq, yhat)

    if not tmp_path.exists():
        model.save(tmp_path)

    trial.set_user_attr("model_path", str(tmp_path))
    trial.set_user_attr("seq_len_used", L)
    return val_rmse

In [ ]:
storageB3 = prepare_journal_storage("ground_trackB_cnnlstm")
studyB3 = optuna.create_study(direction="minimize",
                              sampler=TPESampler(seed=SEED),
                              pruner=MedianPruner(n_startup_trials=8, n_warmup_steps=5),
                              study_name="ground_trackB_cnnlstm",
                              storage=storageB3, load_if_exists=True)
print("Running Study B3 (CNN-LSTM)…")
studyB3.optimize(objective_cnnlstm, n_trials=35, show_progress_bar=True)

best_cnn, _ = _safe_load_best(studyB3)
bestL3 = studyB3.best_trial.user_attrs["seq_len_used"]
Xte_seq, yte_seq, idx_CNN = build_seq_with_idx(Xte_s, yte, bestL3)
yhatB3 = best_cnn.predict(Xte_seq, verbose=0).squeeze()
y_base_CNN = pd.Series(y_base, index=Xte_df.index).reindex(idx_CNN).to_numpy()
print("Best CNN-LSTM params:", studyB3.best_trial.params | {"seq_len": bestL3})
print(f"CNN-LSTM test → RMSE: {_rmse(yte_seq, yhatB3):.4f} | MAE: {mean_absolute_error(yte_seq, yhatB3):.4f} | R2: {r2_score(yte_seq, yhatB3):.4f} | Skill: {skill(yte_seq, yhatB3, y_base_CNN):.3f}")


### Transformer

In [ ]:
def objective_transformer(trial: optuna.Trial) -> float:
    K.clear_session()
    L       = trial.suggest_categorical("seq_len", [6, 12, 18, 24])
    d_model = trial.suggest_categorical("d_model", [32, 64, 96, 128])
    heads   = trial.suggest_categorical("heads", [2, 4, 8])
    if d_model % heads != 0:
        raise optuna.TrialPruned()
    ff_dim  = trial.suggest_categorical("ff_dim", [64, 96, 128, 192])
    att_do  = trial.suggest_float("att_dropout", 0.0, 0.3)
    do      = trial.suggest_float("dropout", 0.0, 0.4)
    lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
    bs      = trial.suggest_categorical("batch", [64, 128, 256])
    eps     = trial.suggest_int("epochs", 40, 120)

    Xtr_seq, ytr_seq = _build_seq(Xtr_s, ytr, L)
    Xva_seq, yva_seq = _build_seq(Xva_s, yva, L)
    if min(map(len,[Xtr_seq, Xva_seq])) == 0: 
        raise optuna.TrialPruned()

    inp = layers.Input(shape=(L, Xtr_seq.shape[2]))
    x   = layers.Dense(d_model)(inp)
    x2  = layers.MultiHeadAttention(num_heads=heads, key_dim=d_model//heads, dropout=att_do)(x, x)
    x   = layers.Add()([x, x2]); x = layers.LayerNormalization()(x)
    ff  = layers.Dense(ff_dim, activation="relu")(x)
    ff  = layers.Dense(d_model)(ff)
    x   = layers.Add()([x, ff]); x = layers.LayerNormalization()(x)
    x   = layers.GlobalAveragePooling1D()(x)
    x   = layers.Dropout(do)(x)
    out = layers.Dense(1)(x)
    model = models.Model(inp, out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"])

    tmp_dir  = ART_DIR / f"B_transformer_t{trial.number:04d}"; tmp_dir.mkdir(parents=True, exist_ok=True)
    tmp_path = (tmp_dir / "best.keras").resolve()

    cbs = [
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=0),
        callbacks.ModelCheckpoint(filepath=str(tmp_path), monitor="val_loss",
                                  save_best_only=True, save_weights_only=False),
        TFKerasPruningCallback(trial, "val_loss"),
    ]
    model.fit(Xtr_seq, ytr_seq, validation_data=(Xva_seq, yva_seq),
              epochs=eps, batch_size=bs, verbose=0, callbacks=cbs)

    yhat = model.predict(Xva_seq, verbose=0).squeeze()
    val_rmse = _rmse(yva_seq, yhat)

    if not tmp_path.exists():
        model.save(tmp_path)

    trial.set_user_attr("model_path", str(tmp_path))
    trial.set_user_attr("seq_len_used", L)
    return val_rmse

In [ ]:
storageB4 = prepare_journal_storage("ground_trackB_transformer")
studyB4 = optuna.create_study(direction="minimize",
                              sampler=TPESampler(seed=SEED),
                              pruner=MedianPruner(n_startup_trials=8, n_warmup_steps=5),
                              study_name="ground_trackB_transformer",
                              storage=storageB4, load_if_exists=True)
print("Running Study B4 (Transformer)…")
studyB4.optimize(objective_transformer, n_trials=40, show_progress_bar=True)

best_tr, _ = _safe_load_best(studyB4)
bestL4 = studyB4.best_trial.user_attrs["seq_len_used"]
Xte_seq, yte_seq, idx_TR = build_seq_with_idx(Xte_s, yte, bestL4)
yhatB4 = best_tr.predict(Xte_seq, verbose=0).squeeze()
y_base_TR = pd.Series(y_base, index=Xte_df.index).reindex(idx_TR).to_numpy()
print("Best Transformer params:", studyB4.best_trial.params | {"seq_len": bestL4})
print(f"Transformer test → RMSE: {_rmse(yte_seq, yhatB4):.4f} | MAE: {mean_absolute_error(yte_seq, yhatB4):.4f} | R2: {r2_score(yte_seq, yhatB4):.4f} | Skill: {skill(yte_seq, yhatB4, y_base_TR):.3f}")


## Best

In [ ]:
best_params = {
    "MLP":        studyA.best_trial.params,
    "LSTM":       studyB1.best_trial.params | {"seq_len": studyB1.best_trial.user_attrs["seq_len_used"]},
    "BiLSTM":     studyB2.best_trial.params | {"seq_len": studyB2.best_trial.user_attrs["seq_len_used"]},
    "CNN_LSTM":   studyB3.best_trial.params | {"seq_len": studyB3.best_trial.user_attrs["seq_len_used"]},
    "Transformer":studyB4.best_trial.params | {"seq_len": studyB4.best_trial.user_attrs["seq_len_used"]},
}
(out := OUT_DIR / "best_hpo_params_all.json")
with open(out, "w") as f:
    json.dump(best_params, f, indent=2)
print("Saved params →", out)

## Visualization

In [ ]:
models_info = {
    "MLP": {
        "type": "tabular",
        "model": best_mlp,
        "idx": Xte_df.index,
        "y_base": y_base
    },
    "LSTM": {
        "type": "seq",
        "model": best_lstm,
        "L": bestL1,
        "idx": idx_LSTM,
        "y_base": y_base_LSTM
    },
    "BiLSTM": {
        "type": "seq",
        "model": best_bi,
        "L": bestL2,
        "idx": idx_BI,
        "y_base": y_base_BI
    },
    "CNN-LSTM": {
        "type": "seq",
        "model": best_cnn,
        "L": bestL3,
        "idx": idx_CNN,
        "y_base": y_base_CNN
    },
    "Transformer": {
        "type": "seq",
        "model": best_tr,
        "L": bestL4,
        "idx": idx_TR,
        "y_base": y_base_TR
    }
}

In [ ]:
rows = []
OUT_FIG = OUT_DIR
for name, cfg in models_info.items():
    print(f"\n=== {name} ===")
    if cfg["type"] == "tabular":
        y_true = yte
        y_pred = cfg["model"].predict(Xte, verbose=0).squeeze()
        idx    = cfg["idx"]
        yb     = cfg["y_base"]
    else:
        L = int(cfg["L"])
        X_seq, y_seq, idx = build_seq_with_idx(Xte_s, yte, L)
        if len(X_seq) == 0:
            print("No hay secuencias válidas (NaNs). Se omite.")
            continue
        y_true = y_seq
        y_pred = cfg["model"].predict(X_seq, verbose=0).squeeze()
        yb     = pd.Series(y_base, index=Xte_df.index).reindex(idx).to_numpy()

    rmse = _rmse(y_true, y_pred)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    skl  = skill(y_true, y_pred, yb)
    print(f"RMSE={rmse:.4f} | MAE={mae:.4f} | R2={r2:.4f} | Skill={skl:.3f}")

    rows.append({"model": name, "RMSE": rmse, "MAE": mae, "R2": r2, "Skill": skl})

    # 1) Serie temporal (recorte)
    N = min(400, len(y_true))
    plt.figure(figsize=(12, 3.6))
    plt.plot(idx[:N], y_true[:N], label="truth", lw=1.4)
    plt.plot(idx[:N], y_pred[:N], label=name, lw=1.1)
    plt.plot(idx[:N], yb[:N], label="baseline", lw=1.0, alpha=0.7)
    plt.title(f"Test — Truth vs {name} vs Baseline ({TARGET})")
    plt.ylabel("GHI (W/m²)" if TARGET.startswith("y_ghi") else "k-index")
    plt.xlabel("Time"); plt.grid(True, ls="--", alpha=0.3); plt.legend()
    plt.xticks(rotation=45); plt.tight_layout()
    plt.savefig(OUT_FIG / f"{name}_ts_test.png", dpi=140)
    plt.show()

    # 2) Scatter
    lim_min = float(min(np.min(y_true), np.min(y_pred)))
    lim_max = float(max(np.max(y_true), np.max(y_pred)))
    plt.figure(figsize=(4.8, 4.8))
    plt.scatter(y_true, y_pred, s=10, alpha=0.5)
    plt.plot([lim_min, lim_max], [lim_min, lim_max], 'r--', lw=1.0)
    plt.xlabel("Actual"); plt.ylabel("Predicted")
    plt.title(f"{name} — Actual vs Predicted\nRMSE={rmse:.3f} MAE={mae:.3f} R2={r2:.3f}")
    plt.grid(True, ls="--", alpha=0.3); plt.tight_layout()
    plt.savefig(OUT_FIG / f"{name}_scatter.png", dpi=140)
    plt.show()

    # 3) Histograma de residuales
    resid = y_pred - y_true
    plt.figure(figsize=(6, 3.2))
    plt.hist(resid, bins=50, alpha=0.85)
    plt.axvline(0, color='r', ls='--', lw=1)
    plt.title(f"{name} — Residuals (mean={np.mean(resid):.3f})")
    plt.xlabel("Residual"); plt.ylabel("Frequency")
    plt.grid(True, ls="--", alpha=0.3); plt.tight_layout()
    plt.savefig(OUT_FIG / f"{name}_residuals.png", dpi=140)
    plt.show()

In [ ]:
results_df = pd.DataFrame(rows).sort_values("RMSE")
print("\n=== Test Summary ===")
print(results_df.round(4))
results_df.to_csv(OUT_DIR / "hpo_models_test_summary.csv", index=False)